## Imports

In [127]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [128]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 26th Mar. 4th


In [129]:
filename = glob.glob('redfin*.csv')
filename = filename[0]

In [130]:
df = pd.read_csv(filename)
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [131]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [133]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

210


In [138]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$772,868


In [139]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$536


In [140]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$162,302,260


## Previous Week's Values

In [24]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: $XX million)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
100
Input Previous Week Condo Average Sales Price:
607764
Input Previous Week Condo Average PSF:
495
Input Previous Week Condo Sales total (ex: $XX million)
60776400


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [141]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [142]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [143]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [144]:
pd.set_option('display.max_columns',None)

In [145]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
77,PAST SALE,February-29-2024,Condo/Co-op,4701 N Meridian Ave #621,Miami Beach,FL,33140.0,6300000.0,3.0,3.5,4701 NORTH MERIDIAN CONDO,3418.0,NaN,2019.0,NaN,1843.0,14158.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11479305,N,Y,25.822813,-80.130733,1,orange


In [146]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [147]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [148]:
m.save('index.html')

## Data snagger

In [90]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [149]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
77,PAST SALE,February-29-2024,Condo/Co-op,4701 N Meridian Ave #621,Miami Beach,FL,33140.0,6300000.0,3.0,3.5,4701 NORTH MERIDIAN CONDO,3418.0,NaN,2019.0,NaN,1843.0,14158.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11479305,N,Y,25.822813,-80.130733,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [150]:
df_top_ten['PRICE'].iloc[0]

6300000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [151]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_03042024


# CREATE TEMPLATE 

In [152]:
muni_set = set(df_top_ten['CITY'])

In [153]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [154]:
df_top_ten.reset_index(inplace=True,drop=True)

In [155]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [156]:
top_sale

'4701 NORTH MERIDIAN CONDO at 4701 N Meridian Ave #621 in Miami Beach'

In [157]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [111]:
# df_top_ten

## RESUME HERE

In [164]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 4701 NORTH MERIDIAN CONDO tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,900,000M to $6,300,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami, Miami Beach, Sunny Isles Beach, Bal Harbour

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 210 condo sales totaling $162,302,260 million from Feb. 26th to Mar. 4th. The previous week, brokers closed 100 condo sales totaling $60,776,400.

Last week’s units sold for an average of $772,868, higher than the $607,764 sales average from the previous week. The average price p

In [165]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [166]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [167]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: 4701 NORTH MERIDIAN CONDO tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,900,000M to $6,300,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami, Miami Beach, Sunny Isles Beach, Bal Harbour

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 210 condo sales totaling $162,302,260 million from Feb. 26th to Mar. 4th. The previous week, brokers closed 100 condo sales totaling $60,776,400.

Last week’s units sold for an average of $772,868, higher than the $607,764 sales average from the previous week. The average price p

In [184]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami-Beach/4701-N-Meridian-Ave-33140/unit-621/home/79136299


In [186]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/15701-Collins-Ave-33160/unit-1003/home/171036175


In [180]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)


https://www.redfin.com/FL/Miami/2811-S-Bayshore-Dr-33133/unit-14C/home/145590769


In [182]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)


https://www.redfin.com/FL/Miami/700-NE-26th-Ter-33137/unit-4801/home/187197565


In [183]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,February-29-2024,Condo/Co-op,4701 N Meridian Ave #621,Miami Beach,FL,33140.0,6300000.0,3.0,3.5,4701 NORTH MERIDIAN CONDO,3418.0,NaN,2019.0,NaN,1843.0,14158.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-N-M...,MARMLS,A11479305,N,Y,25.822813,-80.130733,1,orange,4701 NORTH MERIDIAN CONDO at 4701 N Meridian A...
1,PAST SALE,February-27-2024,Condo/Co-op,5959 Collins Ave #1003,Miami Beach,FL,33140.0,6100000.0,3.0,4.5,THE RESIDENCES AT THE BAT,3640.0,NaN,2005.0,NaN,1676.0,5503.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11292968,N,Y,25.841880,-80.120969,2,blue,THE RESIDENCES AT THE BAT at 5959 Collins Ave ...
2,PAST SALE,February-28-2024,Condo/Co-op,2627 S Bayshore Dr #2501,Miami,FL,33133.0,5750000.0,4.0,5.5,GROVENOR HOUSE CONDO,4026.0,NaN,2006.0,NaN,1428.0,10172.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11474947,N,Y,25.732211,-80.234936,3,blue,GROVENOR HOUSE CONDO at 2627 S Bayshore Dr #25...
3,PAST SALE,February-29-2024,Condo/Co-op,10295 Collins Ave #1909,Bal Harbour,FL,33154.0,5100000.0,3.0,3.5,10295 COLLINS AVE RESDNTA,2643.0,NaN,2007.0,NaN,1930.0,2960.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11500083,N,Y,25.898952,-80.123405,4,blue,10295 COLLINS AVE RESDNTA at 10295 Collins Ave...
4,PAST SALE,February-28-2024,Condo/Co-op,18555 Collins Ave #5403,Sunny Isles Beach,FL,33160.0,4800000.0,3.0,4.5,18555 COLLINS AVENUE COND,3130.0,NaN,2016.0,NaN,1534.0,6172.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11499658,N,Y,25.948195,-80.120410,5,blue,18555 COLLINS AVENUE COND at 18555 Collins Ave...
5,PAST SALE,March-1-2024,Condo/Co-op,2811 S Bayshore Dr Unit 14C,Miami,FL,33133.0,4760000.0,2.0,2.5,C.G.B. SUBDIVISION,1967.0,NaN,2020.0,NaN,2420.0,4335.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2811-S-Bayshor...,MARMLS,A11519060,N,Y,25.728751,-80.238013,6,blue,C.G.B. SUBDIVISION at 2811 S Bayshore Dr Unit ...
6,PAST SALE,February-26-2024,Condo/Co-op,17875 Collins Ave #2001,Sunny Isles Beach,FL,33160.0,4050000.0,3.0,3.5,ACQUALINA OCEAN RESIDENCE,2736.0,NaN,2006.0,NaN,1480.0,5202.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11497457,N,Y,25.941240,-80.120744,7,blue,ACQUALINA OCEAN RESIDENCE at 17875 Collins Ave...
7,PAST SALE,March-1-2024,Condo/Co-op,9701 Collins Ave Unit 1405S,Bal Harbour,FL,33154.0,3500000.0,2.0,2.5,BAL HARBOUR NORTH SOUTH C,1777.0,NaN,2011.0,NaN,1970.0,6410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9701-Col...,MARMLS,A11400848,N,Y,25.888491,-80.122881,8,blue,BAL HARBOUR NORTH SOUTH C at 9701 Collins Ave ...
8,PAST SALE,February-29-2024,Condo/Co-op,15701 Collins Ave #1003,Sunny Isles Beach,FL,33160.0,2900000.0,2.0,2.5,15701 COLLINS CONDO,1605.0,NaN,2020.0,NaN,1807.0,2465.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11479226,N,Y,25.921979,-80.121703,9,blue,15701 COLLINS CONDO at 15701 Collins Ave #1003...
9,PAST SALE,February-27-2024,Condo/Co-op,700 NE 26 Ter #4801,Miami,FL,33137.0,2900000.0,3.0,4.5,Missoni Baia,2552.0,NaN,2023.0,NaN,1136.0,3823.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11442527,N,Y,25.802198,-80.185790,10,blue,Missoni Baia at 700 NE 26 Ter #4801 in Miami


## Time on Market Calculator

In [173]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 11, 15) ## List (Earlier) date
date2 = datetime(2024, 2, 27) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

104
